In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
# 从csv里读取数据到dataframe里去，告诉pandas第一行是标题，另外只要限定的4列
data = pd.read_csv('king_county_data_geocoded.csv', header=0)[['AppraisedValue','SqFtLot','lat','long']]
# 构建样本数据和测试数据
train_data = data[0:50000]
test_data = data[50000:51000]
# pop的含义是将指定列从数据集中抽出来
train_x, train_y = train_data, train_data.pop("AppraisedValue")
test_x, test_y = test_data, test_data.pop("AppraisedValue")
# 归一化，经过调试发现z-score的效果更好。min-max差一点
# train_x = (train_x - train_x.min()) / (train_x.max() - train_x.min())
# test_x = (test_x - test_x.min()) / (test_x.max() - test_x.min())
train_x = (train_x - train_x.mean()) / (train_x.std())
test_x = (test_x - test_x.mean()) / (test_x.std())

In [7]:
# 定义占位符，tensorflow run的时候用到
xtr = tf.placeholder("float", [None, 3])
xte = tf.placeholder("float", [3])
# 使用曼哈顿距离，测试欧氏距离效果差一点
# reduce_sum是降维求和的方法，reduction_indices=1 代表按行求和
distance = tf.reduce_sum(tf.abs(tf.add(xtr, tf.negative(xte))), reduction_indices=1)
#distance = tf.sqrt(tf.reduce_sum(tf.pow(tf.add(xtr,tf.negative(xte)),2),reduction_indices=1))
# 取出距离最近的100个点
pred = tf.nn.top_k(tf.negative(distance),50)

accuracy = 0.
# tensorflow需要的初始化变量
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    # 循环，每次训练集全取，测试集取一行
    for i in range(len(test_x)):
        nn_index = sess.run(pred, feed_dict={xtr: train_x, xte: test_x.iloc[i]})
        print('nn_index',nn_index)
        # 最近的几个点求均值作为预测值
        pre_value = np.mean(train_y[nn_index.indices])
        print("Test", i, "预测值:", pre_value,"真实值:", test_y.iloc[i])
        # 偏差10%以内认为可以接受
        if abs(pre_value /  test_y.iloc[i]-1)<=0.1:
            accuracy += 1. / len(test_x)
    print("结束!")
    print("准确率:", accuracy)

nn_index TopKV2(values=array([-0.09128764, -0.11335918, -0.11662553, -0.11701199, -0.11910459,
       -0.12076434, -0.12857696, -0.13867615, -0.14101286, -0.1425351 ,
       -0.14659932, -0.1501042 , -0.15405703, -0.15965721, -0.1604306 ,
       -0.16293932, -0.16539595, -0.16663328, -0.16683023, -0.16834891,
       -0.16897167, -0.170003  , -0.17066497, -0.17082421, -0.17111135,
       -0.17134261, -0.1740403 , -0.17980573, -0.17998374, -0.18066266,
       -0.18149023, -0.18177009, -0.18408175, -0.19003211, -0.19120756,
       -0.19196546, -0.19233191, -0.19239207, -0.19323193, -0.19419482,
       -0.19433607, -0.19482183, -0.19598956, -0.196502  , -0.19682345,
       -0.1971263 , -0.19742784, -0.197801  , -0.19787058, -0.19957608],
      dtype=float32), indices=array([12332, 49401, 31730,  9228, 47728, 16377, 47196, 35929,  4472,
       46959, 49271, 26972, 47132, 48824, 45694,  6951, 48264, 48426,
       38079, 48777,  1794,  7085, 46850, 47364,  7608, 47079, 47971,
       47353, 49

nn_index TopKV2(values=array([-0.10220353, -0.10398076, -0.11126556, -0.11719461, -0.12432063,
       -0.12974258, -0.1401158 , -0.1411738 , -0.14627033, -0.14696991,
       -0.14804302, -0.15177521, -0.15629494, -0.1565701 , -0.15936072,
       -0.16503392, -0.16547237, -0.16705683, -0.16751304, -0.17202559,
       -0.17724666, -0.17771018, -0.18030617, -0.18038777, -0.1810219 ,
       -0.18105875, -0.1822561 , -0.18287401, -0.18716004, -0.18873216,
       -0.19102791, -0.1913131 , -0.19140552, -0.19194745, -0.19564903,
       -0.19573522, -0.195858  , -0.19653764, -0.1998672 , -0.2002731 ,
       -0.20034339, -0.20462337, -0.20506752, -0.20545365, -0.205649  ,
       -0.20633999, -0.21056905, -0.21205299, -0.21247962, -0.2135511 ],
      dtype=float32), indices=array([11153, 48941, 19202, 47979, 47475, 47991, 49262, 49944, 47188,
       15447, 13101,  1155,  6859, 23728, 48450, 15899, 48189, 25089,
       19864, 32145, 49254, 46785, 46982, 46601, 19309, 47698, 11467,
       19967, 29

nn_index TopKV2(values=array([-0.23905224, -0.2702183 , -0.27090985, -0.2932149 , -0.29512134,
       -0.3216762 , -0.32376575, -0.33188406, -0.33660692, -0.33867794,
       -0.35929322, -0.3673889 , -0.3758783 , -0.37885702, -0.3795597 ,
       -0.3836584 , -0.39072898, -0.40034163, -0.42154548, -0.42512435,
       -0.43760893, -0.43879333, -0.44567573, -0.45086423, -0.4521041 ,
       -0.461698  , -0.4621832 , -0.4634521 , -0.47256184, -0.47655457,
       -0.47987297, -0.48322874, -0.48783123, -0.4882966 , -0.4884935 ,
       -0.4897602 , -0.4906961 , -0.49256748, -0.4938015 , -0.49458325,
       -0.4982646 , -0.498835  , -0.5021234 , -0.5031935 , -0.5032163 ,
       -0.5041706 , -0.51508737, -0.5157424 , -0.5163184 , -0.5202521 ],
      dtype=float32), indices=array([49883, 19972,  8150,  4556, 48921, 48393, 48630,  3649, 47002,
       48676,  5999, 46683, 47765, 46839, 49651, 47328, 47482, 47370,
       24898, 48458, 46956, 47490, 48455, 48670, 46786, 47220, 47549,
       47773, 49

       47473, 47715, 47732, 47435, 10400], dtype=int32))
Test 93 预测值: 387980.0 真实值: 429000
nn_index TopKV2(values=array([-0.20438004, -0.20458895, -0.2046153 , -0.205599  , -0.20617804,
       -0.20674136, -0.20716578, -0.20753938, -0.20813984, -0.20875528,
       -0.2096431 , -0.20983127, -0.20989417, -0.21019733, -0.21038696,
       -0.21045959, -0.21051064, -0.21051803, -0.21142481, -0.21272466,
       -0.21276303, -0.21283361, -0.21290249, -0.21329062, -0.21335341,
       -0.2134828 , -0.21392134, -0.21412617, -0.21415228, -0.21418469,
       -0.2142541 , -0.21507528, -0.21514931, -0.2152841 , -0.21532845,
       -0.21584284, -0.21587904, -0.21605071, -0.21632788, -0.2163718 ,
       -0.2167289 , -0.21678703, -0.21696179, -0.21726793, -0.21733695,
       -0.21756238, -0.21775821, -0.2178042 , -0.21802567, -0.21802771],
      dtype=float32), indices=array([40482, 36725, 36720, 42196, 13277, 40823, 43337, 25179, 38316,
       40047, 14118, 13195, 12765, 17858,  6867, 19914, 10322, 22

nn_index TopKV2(values=array([-0.44843435, -0.46751055, -0.47178307, -0.47246104, -0.51079786,
       -0.51114625, -0.51130736, -0.5114234 , -0.5159483 , -0.5164056 ,
       -0.52084935, -0.52654916, -0.529176  , -0.5301355 , -0.53050643,
       -0.5307241 , -0.53161037, -0.5336622 , -0.5350389 , -0.5360103 ,
       -0.53609896, -0.5361146 , -0.53613573, -0.536724  , -0.53714335,
       -0.53725326, -0.5384369 , -0.5391635 , -0.5395246 , -0.5396956 ,
       -0.539896  , -0.54074824, -0.5407864 , -0.54110825, -0.5422268 ,
       -0.5436563 , -0.5437232 , -0.54387116, -0.5441036 , -0.5443208 ,
       -0.54496014, -0.54507244, -0.5451057 , -0.5463058 , -0.54639107,
       -0.54704183, -0.5473466 , -0.54774404, -0.54802185, -0.5480418 ],
      dtype=float32), indices=array([11480, 26899, 33192, 41472, 11801,  2456, 15298,  2238, 21964,
       14583,  1473, 34879, 17867, 34914,  6418, 10016, 38636, 34156,
       35525, 11351, 14193,  6692,     8, 15067,  5662, 19087,    31,
       13275,   

...

nn_index TopKV2(values=array([-0.37465593, -0.3747654 , -0.37477022, -0.37487894, -0.37492225,
       -0.37500152, -0.37633824, -0.3764867 , -0.37723377, -0.3776738 ,
       -0.38105837, -0.38152924, -0.38155854, -0.38157472, -0.38174835,
       -0.38196325, -0.3823222 , -0.382396  , -0.38257378, -0.3832463 ,
       -0.38451296, -0.38512367, -0.3855975 , -0.38594157, -0.38673994,
       -0.38759464, -0.3876172 , -0.38762367, -0.3885169 , -0.38973215,
       -0.39012343, -0.39075023, -0.3908422 , -0.39138645, -0.3916052 ,
       -0.39160568, -0.39184332, -0.39211875, -0.39229184, -0.39232725,
       -0.39302048, -0.39397144, -0.39405832, -0.3943203 , -0.39468732,
       -0.39501327, -0.39536256, -0.3954238 , -0.3960635 , -0.39620867],
      dtype=float32), indices=array([20888, 18980,  2027, 15158,  8269, 12444, 32315, 34721,  9361,
        8160, 19901, 28917, 29530, 29825, 14634, 32426, 36696, 36128,
       34278, 23510,  9254, 12349, 10846, 10852, 15835, 30794,  7351,
       11680, 34

nn_index TopKV2(values=array([-0.30347475, -0.30382404, -0.30388528, -0.30513123, -0.3057924 ,
       -0.3060805 , -0.30834383, -0.30837137, -0.30899006, -0.30950204,
       -0.310213  , -0.31031793, -0.31042743, -0.31043223, -0.31054094,
       -0.31058425, -0.31066352, -0.31112707, -0.3117223 , -0.31200027,
       -0.3121487 , -0.31289577, -0.31316918, -0.3133358 , -0.31356943,
       -0.31390417, -0.31418425, -0.31430808, -0.3147898 , -0.31480664,
       -0.31513065, -0.3156136 , -0.3156251 , -0.3156298 , -0.31566542,
       -0.31595677, -0.3161903 , -0.3162235 , -0.3164218 , -0.31651598,
       -0.31663272, -0.31665486, -0.31712437, -0.31716672, -0.31744355,
       -0.31744534, -0.31797814, -0.31820044, -0.3182965 , -0.31835276],
      dtype=float32), indices=array([36840, 32900, 34299, 11768, 17782,  6279, 38891, 37907, 38226,
        5919,  3348, 20888, 18980,  2027, 15158,  8269, 12444, 13886,
       15993, 32315, 34721,  9361, 17615,  8160,  3438, 18336, 40119,
        2130,  8

nn_index TopKV2(values=array([-0.3562458 , -0.35844198, -0.35846016, -0.35848653, -0.35993055,
       -0.3603637 , -0.36053124, -0.361171  , -0.3612212 , -0.36190385,
       -0.36455694, -0.36551523, -0.366163  , -0.36617944, -0.36678973,
       -0.36781603, -0.3682837 , -0.36879832, -0.36892286, -0.3689643 ,
       -0.36904627, -0.36908355, -0.36908594, -0.36973968, -0.37002861,
       -0.37075603, -0.37127173, -0.37154642, -0.37168124, -0.37173477,
       -0.3721909 , -0.37266526, -0.37303057, -0.3732605 , -0.37332523,
       -0.37353963, -0.37397698, -0.37454513, -0.3747419 , -0.37487617,
       -0.37547725, -0.37577584, -0.37598225, -0.3759952 , -0.3762141 ,
       -0.37638134, -0.37655053, -0.37689695, -0.37693998, -0.37697   ],
      dtype=float32), indices=array([ 4946, 12523, 39548, 27567, 21142, 14504, 26753, 27200, 22793,
       17739, 17470, 30121, 18197, 33912, 11887, 25517, 23653, 16092,
       13493, 12564, 10569, 10916,  2192, 18455, 29156, 18373,  6891,
       35519, 35

nn_index TopKV2(values=array([-0.22689725, -0.2322506 , -0.23228824, -0.23229241, -0.2326644 ,
       -0.23269887, -0.23873496, -0.23877165, -0.23927277, -0.24443981,
       -0.24893633, -0.25240156, -0.25317368, -0.25391555, -0.25546473,
       -0.25570822, -0.25652766, -0.2565482 , -0.2567866 , -0.25684518,
       -0.25690687, -0.2571123 , -0.25748777, -0.25769836, -0.25783348,
       -0.2583317 , -0.25850427, -0.25852123, -0.2587157 , -0.25931448,
       -0.2594067 , -0.259677  , -0.26055872, -0.2608071 , -0.2614947 ,
       -0.26213005, -0.26304066, -0.26362002, -0.26582292, -0.2659608 ,
       -0.26602393, -0.26782152, -0.26855916, -0.26867974, -0.26903045,
       -0.27015424, -0.27030516, -0.27055284, -0.27055675, -0.27101326],
      dtype=float32), indices=array([ 2247, 39395, 39312, 39776, 40242, 40302,  2592,  8767, 34257,
       15194, 40880, 30079, 26635, 34471, 41556, 41588, 31971,  3261,
       15107, 26450, 28941, 12386, 29203, 28270, 31568, 37216, 27150,
       26773, 35

nn_index TopKV2(values=array([-0.4311267 , -0.4416643 , -0.45194656, -0.4537746 , -0.45395124,
       -0.45556143, -0.4563831 , -0.45923173, -0.46539193, -0.4655239 ,
       -0.46559143, -0.46571335, -0.46622157, -0.46758437, -0.46849784,
       -0.4709672 , -0.47103614, -0.47113654, -0.4712768 , -0.47139025,
       -0.4721174 , -0.47230527, -0.47325364, -0.47398096, -0.4740318 ,
       -0.4743191 , -0.47493845, -0.47556505, -0.475757  , -0.47589982,
       -0.47647953, -0.47720551, -0.477384  , -0.47791725, -0.47819403,
       -0.47917014, -0.47920278, -0.47962832, -0.47993127, -0.48026484,
       -0.48056954, -0.48065937, -0.48079023, -0.48093575, -0.48118657,
       -0.48155978, -0.48167354, -0.48205084, -0.48231012, -0.48275715],
      dtype=float32), indices=array([16461, 25747,  3830,  3287, 15490,  8828,   673, 25416, 34771,
        8224,  3024,  2928,  5413,  1531,  2898, 10960, 10644, 49104,
       16943,  6388, 31193, 36251, 37555, 24864, 22851, 29521,  1590,
        2645, 21